In [6]:
import spotipy
import pandas as pd

# Authentication with Spotify

In [7]:
# Client Flow - to get features and analysis from tracks
# Authenticate my user from Spotify

CLIENT_ID = "5424fe3bb2b646c8a2144d03d41f22c7"
CLIENT_SECRET = "f13ad8573f3444908afdf6a6199db827"
token = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=token)

# Load dataset with ratings and features

In [8]:
df_toptracks_features = pd.read_csv("./dataset/toptracks_ratings_features.csv")
df_toptracks_features

,id,name,range,rating,id_copy,energy,danceability,loudness,valence,tempo,mode,speechiness,acousticness,instrumentalness,duration_ms,time_signature
0,7L5IwfKB6W0tadcSh9wlyH,Ouverture,short_term,5.393939,7L5IwfKB6W0tadcSh9wlyH,0.3360,0.4960,-19.440,0.1850,129.006,1,0.0513,0.95500,0.912000,322173,4
1,0tAZi3X7dUdd7m8OXB8pMA,Shadow,short_term,5.272727,0tAZi3X7dUdd7m8OXB8pMA,0.2750,0.0832,-15.256,0.0334,170.316,0,0.0347,0.88700,0.853000,558267,1
2,1XMDIKQbV30WJPKLMN6MKv,INSTRUCTION,short_term,4.969697,1XMDIKQbV30WJPKLMN6MKv,0.3550,0.4910,-12.480,0.0369,145.405,1,0.0354,0.38200,0.887000,262733,4
3,1XZdwzd8DTDvkjVc0eJ9BI,Wildlife Analysis,short_term,4.909091,1XZdwzd8DTDvkjVc0eJ9BI,0.0204,0.1550,-31.212,0.2780,79.755,0,0.0462,0.99500,0.936000,75627,4
4,1f4cKwcKfNiLbQr8x2tZ3C,Melt!,short_term,4.848485,1f4cKwcKfNiLbQr8x2tZ3C,0.9190,0.7850,-13.059,0.4430,131.037,1,0.0552,0.00772,0.916000,214307,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,2OqtZbITDWCFUHAT9fmdin,Choses nouvelles,long_term,4.242424,2OqtZbITDWCFUHAT9fmdin,0.3830,0.6690,-12.993,0.5690,140.011,0,0.0385,0.46500,0.856000,234600,4
234,6EPRKhUOdiFSQwGBRBbvsZ,Ace of Spades,long_term,4.181818,6EPRKhUOdiFSQwGBRBbvsZ,0.9100,0.3420,-7.748,0.5470,140.452,1,0.0758,0.00004,0.000121,166360,4
235,4Jj8pWRyVjh0KIJLrcreRa,Here We Go Jack,long_term,4.121212,4Jj8pWRyVjh0KIJLrcreRa,0.3090,0.6790,-10.606,0.4730,163.979,1,0.0320,0.58500,0.907000,237938,4
236,3v9g8iM3v5irWQHqFWaDSo,Cómo Te Quiero,long_term,4.060606,3v9g8iM3v5irWQHqFWaDSo,0.2770,0.6080,-11.917,0.4370,126.501,1,0.0282,0.62200,0.896000,242536,4


# Audio analysis

In [ ]:
def analyze_analysis(playlist, df_features):
    #Dicts specifying the analysis features we want to extract, and a dict to store them in. 
    #The is dynamic so audio features can be added if wanted. Complete list avaliable at:
    #https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/
    
    analysis_features_gathered = {}
    analysis_feature_list = {
        "segments" : [
            "pitches",
            "timbre"
        ],
        "sections" : [
            "key",
            "mode"
        ]
    }

    #Colunm names and framwork for our final dataframe
    framework = []
    for i, (key, val) in enumerate(analysis_feature_list.items()):
        for idx in range(len(val)):
            framework.append(val[idx])
    df_framework = pd.DataFrame(columns=framework)

    for count, track in enumerate(playlist):
        #First store the track duration in milliseconds
        track_dur = df_features["duration_ms"][count]
        
        #gather the desired values (expressed in the analysis_feature_list) from each track 
        #and store them in our new analysis_features_gathered dictionary.
        audio_analysis = sp.audio_analysis(track["track"]["id"])
        
        for i, (key, val) in enumerate(analysis_feature_list.items()):
            for item in range(len(val)):
                #Initialize the key to the correct type. Always store within list.
                analysis_features_gathered[val[item]] = []  
                for x, (y, z) in enumerate(audio_analysis.items()):
                    if y == key:
                        for w in range(len(z)):
                            analysis_features_gathered[val[item]].append(z[w][val[item]])
                            
        #Now we want to configure and calculate different kinds of averages of the lists and numbers
        #we have gathered per segment and section of each track.
        for i, (key, val) in enumerate(analysis_features_gathered.items()):
            p = []
            if type(val[0]) == list: #and is therefore ONLY either timbre or pitch in the segment key. else it would be an int or float.
                for w in range(len(val[0])):
                    store = []
                    for item in val:
                        store.append(item[w])
                    p.append(round(sum(store) / len(store), 2))
                analysis_features_gathered[key] = []
                analysis_features_gathered[key].append(p)
            else:
                if key == "key":
                    #precentage value indicating how many key changes happen in the track, based on its duration.
                    val = [k for k, g in itertools.groupby(val)] #remove consecutive duplicate values.
                    ms_per_chord_change = track_dur/len(val) #how many miliseconds per chord change on average.
                    change_percentage = (100/track_dur)*ms_per_chord_change
                    p.append(abs(change_percentage-100)) #invert values for a more intuitive reading.
                    analysis_features_gathered[key] = p
                else:
                    p.append(round(sum(val)/len(val), 2))
                    analysis_features_gathered[key] = p

        #store and concatonate in dataframes
        df_analysis_features = pd.DataFrame(analysis_features_gathered)
        df_framework = pd.concat([df_framework, df_analysis_features], ignore_index=True)
    
    return df_framework

In [24]:
# all track ids in a list 
track_ids = list(df_toptracks_features["id"])

# a dict to gather the analysis features.
analysis_features_gathered = {}

# a dict specifying which analysis features we want.
analysis_features_list = {
    "segments" : [
        "pitches",
        "timbre"
    ],
     "sections" : [
        "key",
        "mode"
    ]
}

In [29]:
for track_id in track_ids[:2]:
    #gather the analysis features expressed in analysis_features_list from each track 
    #and store them in analysis_features_gathered dictionary.
    
    # spotify API call
    audio_analysis = sp.audio_analysis(track_id)
    
    # for every item in the feaures list. val = ["pitches", "timbre"]. Key = "sections", "segments"
    for i, (key, val) in enumerate(analysis_features_list.items()):
        
        # for feature in val
        for item in range(len(val)):
            #Initialize the key to the correct type.
            analysis_features_gathered[val[item]] = []  
            
            for x, (y, z) in enumerate(audio_analysis.items()):
                if y == key:
                    for w in range(len(z)):
                        analysis_features_gathered[val[item]].append(z[w][val[item]])

In [30]:
print(len(analysis_features_gathered))
print(len(analysis_features_gathered["pitches"]))
print(analysis_features_gathered.keys())

4
1642
dict_keys(['pitches', 'timbre', 'key', 'mode'])


In [36]:
for i, (feature_parent, feature_array) in enumerate(analysis_features_list.items()):
    print("i", i)
    print("key", feature_parent)
    print("val", feature_array)

i 0
key segments
val ['pitches', 'timbre']
i 1
key sections
val ['key', 'mode']


In [35]:
audio_analysis = sp.audio_analysis(track_ids[0])
for count, (feature_category, analysis_items) in enumerate(audio_analysis.items()):
    print("count:", count) # x
    print("info:", feature_category) # y
    print("items:", analysis_items) # z


count: 0
info: meta
items: {'analyzer_version': '4.0.0', 'platform': 'Linux', 'detailed_status': 'OK', 'status_code': 0, 'timestamp': 1663795616, 'analysis_time': 7.11589, 'input_process': 'libvorbisfile L+R 44100->22050'}
count: 1
info: track
items: {'num_samples': 7103904, 'duration': 322.17252, 'sample_md5': '', 'offset_seconds': 0, 'window_seconds': 0, 'analysis_sample_rate': 22050, 'analysis_channels': 1, 'end_of_fade_in': 0.90063, 'start_of_fade_out': 312.46512, 'loudness': -19.44, 'tempo': 129.006, 'tempo_confidence': 0.775, 'time_signature': 4, 'time_signature_confidence': 0.961, 'key': 11, 'key_confidence': 0.044, 'mode': 1, 'mode_confidence': 0.169, 'codestring': 'eJxdnVmW5CgUQ7eSSzAz7H9jrSvhiKj-qVMmbIwZpDcIcu1WnvP0-ff8lb5naWc9f3v2v_7sp5dy-t859W-fMvd-1vorpeqGU8t82j5_Rf-5l3uojtb7X9GDz1Al9a-Muf_aLrOfpZeUeSY_79P3evpf1X__Wj217jnKXy16ej5VxaXNv1pX_RvznNn3WH-1H15Vdp_10a-zqm1zzbGfMv7qGnrdU_Yz2zz7r56lV_Uyntb2_GuP6lL1u5e5ut75PH9r17ZOUTNbV136imc0Wt3G8IvLOXr4r839UPXSk2PpWh12q37UkP6036p7q

In [ ]:
#Colunm names and framwork for our final dataframe
framework = []
for i, (key, val) in enumerate(analysis_features_list.items()):
    for idx in range(len(val)):
        framework.append(val[idx])

df_framework = pd.DataFrame(columns=framework)
df_framework